In [107]:
import pandas as pd
import numpy as np

In [108]:
# Load the dataset
data = pd.read_csv('HIGGS_train.csv')

/var/folders/cl/vqr669317693_lc6y70yn19r0000gn/T/ipykernel_12315/4104986903.py:2: DtypeWarning: Columns (8,21) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('HIGGS_train.csv')


In [109]:
# Find columns with non-float data types
non_float_columns = data.select_dtypes(exclude=['float']).columns.tolist()

# Convert non-float columns to numeric data type
for col in non_float_columns:
    data[col] = pd.to_numeric(data[col], errors='coerce')

# Find rows with non-float values
nonfloat_rows = data[data.isnull().any(axis=1)]

# Print the row numbers and column names with non-float values
for index, row in nonfloat_rows.iterrows():
    nonfloat_columns = row.index[row.isnull()].tolist()
    print(f"Row {index} contains non-float values in columns: {nonfloat_columns}")

Row 260691 contains non-float values in columns: ['0.00E+00.1']
Row 261025 contains non-float values in columns: ['-1.09E+00']
Row 490958 contains non-float values in columns: ['3.10E+00']
Row 490959 contains non-float values in columns: ['3.10E+00']


In [110]:
# Delete rows with non-float values
data.drop(nonfloat_rows.index, inplace=True)

In [111]:
# Calculate z-scores for each column
z_scores = np.abs((data - data.mean()) / data.std())

# Define threshold for z-scores
z_threshold = 2

# Remove rows with z-score greater than threshold in any column
outlier_rows = data[(z_scores > z_threshold).any(axis=1)]
data = data[(z_scores <= z_threshold).all(axis=1)]

# Print number of outlier rows removed
print(f"{len(outlier_rows)} outlier rows removed.")

261922 outlier rows removed.


In [112]:
# Import necessary libraries
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Split the data into features and target variable
X = data.iloc[:, 1:]
y = data.iloc[:, 0]

#shuffle the data
from sklearn.utils import shuffle
X, y = shuffle(X, y, random_state=0)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)



# Create the logistic regression model
logistic_regression_model = LogisticRegression()

# Fit the model on the training data
logistic_regression_model.fit(X_train, y_train)

# Make predictions on the testing data
predictions = logistic_regression_model.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)


Accuracy: 0.6500069018556132


In [113]:
from sklearn.model_selection import KFold

# apply k-fold cross validation
k_fold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = []

for train_indices, test_indices in k_fold.split(X):
    # split data into training and testing sets
    X_train, X_test = X.iloc[train_indices], X.iloc[test_indices]
    y_train, y_test = y.iloc[train_indices], y.iloc[test_indices]
    
    # create and train the model
    model = LogisticRegression()
    model.fit(X_train, y_train)
    
    # evaluate the model on the testing set
    score = model.score(X_test, y_test)
    scores.append(score)

# print the accuracy of each fold
print(scores)

# print the average accuracy of all 10 folds
print(np.mean(scores))


/opt/homebrew/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/homebrew/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i 

[0.6525970184571699, 0.6483968291528632, 0.6483376715570279, 0.6520543082793504, 0.651640192859467, 0.6492442393587127, 0.6509598603839442, 0.6457242582897034, 0.649835832815689, 0.6508415416925489]
0.6499631752846476


In [114]:
#perform regularized logistic regression on different values of C
from sklearn.linear_model import LogisticRegression

# create a list of C values
C_values = [0.001, 0.01, 0.1, 1, 10, 100, 1000]

# create a list to store the models
models = []

# train one model for each C value
for C in C_values:
        # create and train the model
        model = LogisticRegression(C=C)
        model.fit(X_train, y_train)
        
        # add the model to the list of models
        models.append(model)

# print the number of models
print(len(models))

# print the accuracy for each model
for i in range(len(C_values)):
        print("C={:.3f}, accuracy={:.4f}".format(C_values[i], models[i].score(X_test, y_test)))




/opt/homebrew/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/homebrew/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i 

7
C=0.001, accuracy=0.6238
C=0.010, accuracy=0.6462
C=0.100, accuracy=0.6502
C=1.000, accuracy=0.6508
C=10.000, accuracy=0.6508
C=100.000, accuracy=0.6508
C=1000.000, accuracy=0.6509


/opt/homebrew/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [115]:
#perform xgboost on the dataset
import xgboost as xgb

# create the DMatrix
dtrain = xgb.DMatrix(X_train, label=y_train)

# create the parameter dictionary
params = {'max_depth': 2, 'eta': 1, 'objective': 'binary:logistic'}

# train the model
num_rounds = 2
bst = xgb.train(params, dtrain, num_rounds)

# create the DMatrix for the testing data
dtest = xgb.DMatrix(X_test)

# make predictions
predictions = bst.predict(dtest)

# print the first 4 predictions
print(predictions[:4])

# print the first 4 actual labels
print(y_test[:4])

# print the accuracy
accuracy = accuracy_score(y_test, predictions.round())
print("Accuracy:", accuracy)

[0.4718679 0.4375873 0.4718679 0.4375873]
479793    0.0
401597    0.0
399550    1.0
424447    1.0
Name: 1.00E+00, dtype: float64
Accuracy: 0.6585914159789392
